# EEG alignment

In [2]:
from __future__ import division
import csv
import numpy as np
import matplotlib.pylab as plt
from scipy.signal import resample
import mne
import pandas as pd
import seaborn as sns
from scipy.signal import hilbert
#from utils import circular_histogram
import pycircstat
import os
from datetime import datetime

In [3]:
%run utlis.ipynb

In [114]:
def data_preprocessing(patient_id,channel):
    patient_data=str(patient_id);

    EEG_dir='D:/IM 680/byteflies_dots/'+patient_data+'/EEG File'
    EEG_dir=EEG_dir.replace('/', '\\')
    print(EEG_dir)


    EEG1E_data = pd.read_csv( os.path.join(EEG_dir, 'BF'+patient_data+'_1'+channel+'_EEG1_im680.csv')) 
    EEG1E2_data = pd.read_csv( os.path.join(EEG_dir,'BF'+patient_data+'_2'+channel+'_EEG1_im680.csv')) 
    EEG2E_data = pd.read_csv( os.path.join(EEG_dir, 'BF'+patient_data+'_1'+channel+'_EEG2_im680.csv')) 
    EEG2E2_data = pd.read_csv( os.path.join(EEG_dir,'BF'+patient_data+'_2'+channel+'_EEG2_im680.csv')) 

    IMU_dir='D:/IM 680/byteflies_dots/'+patient_data+'/IMU File'
    IMU_dir=IMU_dir.replace('/', '\\')
    ACC_data = pd.read_csv( os.path.join(IMU_dir, 'BF'+patient_data+'_1'+channel+'_ACC_im680.csv')) 
    ACC2_data = pd.read_csv( os.path.join(IMU_dir,'BF'+patient_data+'_2'+channel+'_ACC_im680.csv')) 
    GRY_data = pd.read_csv( os.path.join(IMU_dir, 'BF'+patient_data+'_1'+channel+'_GYR_im680.csv')) 
    GRY2_data = pd.read_csv( os.path.join(IMU_dir,'BF'+patient_data+'_1'+channel+'_GYR_im680.csv'))
    #find common time for all EEG GRY and ACC data
    start_time=max(ACC_data['time'][0],GRY_data['time'][0],
               ACC2_data['time'][0],GRY2_data['time'][0],
               EEG1E_data['time'][0],EEG1E2_data['time'][0],
               EEG2E_data['time'][0],EEG2E2_data['time'][0])

    end_time = min( ACC_data['time'].values[-1],  GRY_data['time'].values[-1],
                ACC2_data['time'].values[-1], GRY2_data['time'].values[-1],
                EEG1E_data['time'].values[-1],EEG1E2_data['time'].values[-1],
                EEG2E_data['time'].values[-1],EEG2E2_data['time'].values[-1])

    step=0.01;sfreq=int(1/step);
    
    time=np.arange(start=start_time,stop=end_time,step=step)
    #apply conversion factor for all the data
    #resample the data inbetween the start_time and end_time with step
    #filter the resampled data with sampling frequency 1/step and h_freq=0.1 and l_freq=40

    
    GRY_factor=0.0305;EEG_factor=0.4808;ACC_factor=0.0002;

    GRY_1EX = mne.filter.filter_data(data= resample(x=GRY_factor*GRY_data['channel1'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    GRY_1EY = mne.filter.filter_data(data= resample(x=GRY_factor*GRY_data['channel2'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    GRY_1EZ = mne.filter.filter_data(data= resample(x=GRY_factor*GRY_data['channel3'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    GRY_2EX = mne.filter.filter_data(data= resample(x=GRY_factor*GRY2_data['channel1'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    GRY_2EY = mne.filter.filter_data(data= resample(x=GRY_factor*GRY2_data['channel2'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    GRY_2EZ = mne.filter.filter_data(data= resample(x=GRY_factor*GRY2_data['channel3'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)

    ACC_1EX = mne.filter.filter_data(data= resample(x=ACC_factor*ACC_data['channel1'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    ACC_1EY = mne.filter.filter_data(data= resample(x=ACC_factor*ACC_data['channel2'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    ACC_1EZ = mne.filter.filter_data(data= resample(x=ACC_factor*ACC_data['channel3'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    ACC_2EX = mne.filter.filter_data(data= resample(x=ACC_factor*ACC2_data['channel1'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    ACC_2EY = mne.filter.filter_data(data= resample(x=ACC_factor*ACC2_data['channel2'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    ACC_2EZ = mne.filter.filter_data(data= resample(x=ACC_factor*ACC2_data['channel3'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)

    EEG_1E1 = mne.filter.filter_data(data= resample(x=EEG_factor*EEG1E_data['channel'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    EEG_1E2 = mne.filter.filter_data(data= resample(x=EEG_factor*EEG1E_data['channel'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    EEG_2E1 = mne.filter.filter_data(data= resample(x=EEG_factor*EEG1E_data['channel'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)
    EEG_2E2 = mne.filter.filter_data(data= resample(x=EEG_factor*EEG1E_data['channel'],num=len(time), t=time)[0], sfreq=sfreq, h_freq=0.1 , l_freq=40)

    d = {'time':time, 'GRY 1EX':GRY_1EX, 'GRY 1EY':GRY_1EY,'GRY 1EZ':GRY_1EZ,
                  'GRY 2EX':GRY_2EX, 'GRY 2EY':GRY_2EY,'GRY 2EZ':GRY_2EZ,
                  'ACC 1EX':ACC_1EX, 'ACC 1EY':ACC_1EY,'ACC 1EZ':ACC_1EZ,
                  'ACC 2EX':ACC_2EX, 'ACC 2EY':ACC_2EY,'ACC 2EZ':ACC_2EZ,
    'EEG 1E1':EEG_1E1 ,'EEG 1E2':EEG_1E2,'EEG 2E1': EEG_2E1,'EEG 2E2': EEG_2E2
    }
    df = pd.DataFrame(data=d)

    #read the starting time point for PSG and BFDS data
    path='D:\IM 680\supporting_documents'
    PSG_SleepStage_Summary=pd.read_excel(path+'\IM680_PSG_SleepStage_Summary_FINAL.xlsx')
    PSG_StartTimestamps=pd.read_excel(path+'\IM680_BFSD_PSG_StartTimestamps_DateShifted_FINAL.xlsx')
    PSG_SleepStage_Data=pd.read_excel(path+'\IM680_PSG_SleepStage_Data_FINAL.xlsx')

    Data_pd=PSG_SleepStage_Data[PSG_SleepStage_Data['SUBID']==patient_id]
    print(len(Data_pd))
    if len(Data_pd)==0:
        print('There is no sleep stage data for this patient: '+patient_data)
        return df
    StartTime_1351=PSG_StartTimestamps[PSG_StartTimestamps['SUBID']==patient_id].values[0][1]

    StartTime_1351_PSG=PSG_StartTimestamps[PSG_StartTimestamps['SUBID']==patient_id].values[0][3]
    #calculate the difference between PSG adn BFDS data, we know
    time_format = '%Y-%m-%d %H:%M:%S.%f'
    BFSD_time = datetime.utcfromtimestamp(int(StartTime_1351)).strftime(time_format)
    PSG_time = datetime.utcfromtimestamp(int(StartTime_1351_PSG)).strftime(time_format)
    print(BFSD_time+' : This the time on Start of BFSD file.' )
    print(PSG_time +' : This the time on the PSG.')

    TimeDifference=(datetime.utcfromtimestamp(int(StartTime_1351_PSG))-datetime.utcfromtimestamp(int(StartTime_1351))).seconds

    start=df['time'][0]
    df['time']=df['time'].apply(lambda x: x-start)

    print('we need to throw out the first '+ str( TimeDifference)+'s in the BFDS file')

    df=df.iloc[int(TimeDifference/step):,:].reset_index()
    
    start=np.min(df['time'])
    df['time']=df['time'].apply(lambda x: x-start)

    df.rename(columns={'index':'sleep stage'},inplace=True)
    
    stage_list=np.repeat(Data_pd['Stage'].values, 30*100)

    stage_code_list=np.repeat(Data_pd['Stage_Coded'].values, 30*100)
    
    df['sleep stage']=np.nan
    #we also need to drop the later time when we don't have PSG data anymore
    df['sleep stage'].iloc[:len(stage_list)]=stage_list

    df.dropna(inplace=True)

    df['Sleep stage code']=stage_code_list
    return df
    #df.to_csv('BFDS'+patient_data+'.csv')

In [74]:
patient_id=1351;channel='E';

In [72]:
df1351=data_preprocessing(patient_id,channel)

D:\IM 680\byteflies_dots\1351\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

2021-08-20 22:24:53.000000 : This the time on Start of BFSD file.
2021-08-21 01:22:40.000000 : This the time on the PSG.
we need to throw out the first 10667s in the BFDS file


In [51]:
df1351.to_csv('D:/IM 680/byteflies_dots/'+str(patient_id)+'.csv')

In [75]:
df2510=data_preprocessing(2510,'D')

D:\IM 680\byteflies_dots\2510\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

2021-01-25 20:21:01.000000 : This the time on Start of BFSD file.
2021-01-26 02:20:47.000000 : This the time on the PSG.
we need to throw out the first 21586s in the BFDS file


In [77]:
df2510.to_csv('D:/IM 680/byteflies_dots/'+str(2510)+'.csv')

In [78]:
df2939=data_preprocessing(2939,'E')

D:\IM 680\byteflies_dots\2939\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

2021-04-20 22:02:51.000000 : This the time on Start of BFSD file.
2021-04-21 02:22:16.000000 : This the time on the PSG.
we need to throw out the first 15565s in the BFDS file


In [79]:
df2939.to_csv('D:/IM 680/byteflies_dots/'+str(2939)+'.csv')

In [81]:
df4301=data_preprocessing(4301,'D')

D:\IM 680\byteflies_dots\4301\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

2020-11-13 20:09:13.000000 : This the time on Start of BFSD file.
2020-11-14 01:49:11.000000 : This the time on the PSG.
we need to throw out the first 20398s in the BFDS file


In [82]:
df4301.to_csv('D:/IM 680/byteflies_dots/'+str(4301)+'.csv')

In [83]:
df5852=data_preprocessing(5852,'C')

D:\IM 680\byteflies_dots\5852\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

2020-12-01 22:45:55.000000 : This the time on Start of BFSD file.
2020-12-02 02:27:23.000000 : This the time on the PSG.
we need to throw out the first 13288s in the BFDS file


In [84]:
df5852.to_csv('D:/IM 680/byteflies_dots/'+str(5852)+'.csv')

In [88]:
df5995=data_preprocessing(5995,'D')

D:\IM 680\byteflies_dots\5995\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

2021-09-24 22:29:34.000000 : This the time on Start of BFSD file.
2021-09-25 01:18:11.000000 : This the time on the PSG.
we need to throw out the first 10117s in the BFDS file


In [110]:
df7257=data_preprocessing(7257,'D')

D:\IM 680\byteflies_dots\7257\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

2021-05-07 20:52:47.000000 : This the time on Start of BFSD file.
2021-05-08 02:12:19.000000 : This the time on the PSG.
we need to throw out the first 19172s in the BFDS file


In [112]:
df7257.to_csv('D:/IM 680/byteflies_dots/'+str(7257)+'.csv')

note that for patient 7838, I change the file names to be channel A so it is easier for me to read the files in

In [116]:
df7838=data_preprocessing(7838,'A')

D:\IM 680\byteflies_dots\7838\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

904
2021-05-28 23:29:11.000000 : This the time on Start of BFSD file.
2021-05-29 01:29:44.000000 : This the time on the PSG.
we need to throw out the first 7233s in the BFDS file


In [118]:
df7838.to_csv('D:/IM 680/byteflies_dots/'+str(7838)+'.csv')

In [119]:
df7844=data_preprocessing(7844,'E')

D:\IM 680\byteflies_dots\7844\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

890
2021-07-16 20:58:57.000000 : This the time on Start of BFSD file.
2021-07-17 02:06:49.000000 : This the time on the PSG.
we need to throw out the first 18472s in the BFDS file


In [120]:
df7844.to_csv('D:/IM 680/byteflies_dots/'+str(7844)+'.csv')

In [90]:
patient_id=5995;channel='D';

In [121]:
df5995=data_preprocessing(patient_id,channel)

D:\IM 680\byteflies_dots\5995\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

1059
2021-09-24 22:29:34.000000 : This the time on Start of BFSD file.
2021-09-25 01:18:11.000000 : This the time on the PSG.
we need to throw out the first 10117s in the BFDS file


In [123]:
df5995.to_csv('D:/IM 680/byteflies_dots/'+str(5995)+'.csv')

In [124]:
df8898=data_preprocessing(8898,'D')

D:\IM 680\byteflies_dots\8898\EEG File
Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

Setting up band-stop filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 1

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (33.010 sec)

873
2020-07-24 22:34:01.000000 : This the time on Start of BFSD file.
2020-07-25 02:45:42.000000 : This the time on the PSG.
we need to throw out the first 15101s in the BFDS file


In [125]:
df8898.to_csv('D:/IM 680/byteflies_dots/'+str(8898)+'.csv')